In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8937,2024-05-21,Eua Nba,21:00,Boston Celtics,Indiana Pacers,1.23,4.52,221.5,1.84,2.00,-10.5,2.02,10.50,UksVHNdm,0.813008,0.221239,0.543478,0.500000,0.034247,0.6,0.6,NaN,NaN,147.374,20.336518,0.137993,155.284,8.173083,0.052633,149.260,230.350,172.92,146.28,0.0,0.0,0.0,0.0,0.809176,0.058926,0.957870,0.69,0.138,1.666667,0.823247,0.8,-0.023247,-2.98,-0.596,-5.906040,0.586684,0.4,-0.186684
8938,2024-05-21,Alemanha Bbl,13:30,Bonn,Alba Berlin,2.43,1.57,162.5,1.87,1.91,2.5,2.03,1.33,KUxtKJll,0.411523,0.636943,0.534759,0.523560,0.048465,0.4,0.6,NaN,NaN,136.192,17.240104,0.126587,137.164,29.216580,0.213005,145.906,144.832,130.40,107.73,0.0,0.0,0.0,0.0,0.304056,0.014965,0.294628,-0.53,-0.106,-13.490566,0.692040,0.7,0.007960,0.35,0.070,8.142857,0.000000,0.0,0.000000
8939,2024-05-21,Alemanha Bbl,15:30,Vechta,Chemnitz,1.87,1.94,161.5,1.81,1.93,-1.5,1.99,2.06,IchzuDae,0.534759,0.515464,0.552486,0.518135,0.050223,0.6,0.8,NaN,NaN,169.620,53.516175,0.315506,169.810,92.827505,0.546655,147.544,180.954,151.38,157.17,0.0,0.0,0.0,0.0,0.025983,0.045376,0.024443,-2.23,-0.446,-1.950673,0.621257,0.5,-0.121257,-0.44,-0.088,-10.681818,0.560576,0.7,0.139424
8940,2024-05-21,Argentina Liga A,21:00,Independiente de Oliva,Platense,1.82,1.90,156.5,1.85,1.85,-1.5,1.94,2.07,j320vtII,0.549451,0.526316,0.540541,0.540541,0.075766,0.2,0.6,NaN,NaN,177.870,59.504868,0.334541,148.388,22.779600,0.153514,143.102,190.208,118.30,163.53,0.0,0.0,0.0,0.0,0.030413,0.000000,0.045847,-1.41,-0.282,-2.907801,0.566427,0.4,-0.166427,-1.38,-0.276,-3.260870,0.483294,0.5,0.016706
8941,2024-05-21,Argentina Liga A,21:30,La Union,Obras,1.76,1.97,164.5,1.84,1.86,-2.5,2.01,2.39,WWrd6va6,0.568182,0.507614,0.543478,0.537634,0.075796,0.8,0.6,NaN,NaN,212.366,86.670388,0.408118,194.868,35.430534,0.181818,200.222,204.420,157.32,147.00,0.0,0.0,0.0,0.0,0.079621,0.007644,0.122137,-0.14,-0.028,-27.142857,0.499516,0.4,-0.099516,-2.35,-0.470,-2.063830,0.458953,0.4,-0.058953
8942,2024-05-21,Colômbia Lbp,21:40,Piratas de Bogota,Titanes de Barranquilla,1.59,2.26,150.5,1.85,1.85,-4.5,2.01,1.90,IePyLono,0.628931,0.442478,0.540541,0.540541,0.071409,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,94.40,0.0,0.0,0.0,1.0,0.246110,0.000000,0.039786,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8943,2024-05-21,Eua Wnba,23:00,Los Angeles Sparks F,Washington Mystics F,1.63,2.37,158.5,1.81,2.00,-3.5,1.92,2.93,IRBIWoFM,0.613497,0.421941,0.552486,0.500000,0.035438,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.261630,0.070525,0.294506,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8944,2024-05-21,Finlândia Korisliiga,12:30,BC Nokia,Helsinki Seagulls,1.72,2.05,155.5,1.85,1.89,-2.5,1.97,2.45,212U2JdJ,0.581395,0.487805,0.540541,0.529101,0.069200,0.2,0.0,NaN,NaN,165.702,18.601529,0.112259,132.690,28.537253,0.215067,183.514,132.690,176.80,102.00,0.0,0.0,0.0,0.0,0.123791,0.015125,0.153580,3.62,0.724,0.994475,0.000000,0.0,0.000000,0.46,0.092,11.413043,0.718366,0.6,-0.118366
8945,2024-05-21,Israel Superliga,13:00,Ironi Kiryat Ata,Maccabi Ironi Ramat Gan,2.19,1.65,168.5,1.86,1.80,1.5,2.02,1.49,04nLzDyn,0.456621,0.606061,0.537634,0.555556,0.062682,0.2,0.4,NaN,NaN,203.244,69.949404,0.344165,201.416,136.077741,0.675605,198.140,252.458,275.12,440.86,0.0,0.0,0.0,0.0,0.198874,0.023184,0.213542,2.45,0.490,2.428571,0.000000,0.0,0.000000,2.91,0.582,1.116838,0.000000,0.0,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Boston Celtics,Indiana Pacers,221.5,1.84,1.0000,Over
1,15:30,Alemanha Bbl,Vechta,Chemnitz,161.5,1.81,0.9999,Over
2,21:00,Argentina Liga A,Independiente de Oliva,Platense,156.5,1.85,1.0000,Over
3,21:30,Argentina Liga A,La Union,Obras,164.5,1.84,1.0000,Over
4,21:40,Colômbia Lbp,Piratas de Bogota,Titanes de Barranquilla,150.5,1.85,0.9915,Over
5,23:00,Eua Wnba,Los Angeles Sparks F,Washington Mystics F,158.5,1.81,1.0000,Over
6,12:30,Finlândia Korisliiga,BC Nokia,Helsinki Seagulls,155.5,1.85,1.0000,Over
7,20:00,Venezuela Superliga,Llaneros,Broncos,157.5,1.83,0.9996,Over
